In [1]:
'''
Windows: Open Anaconda prompt
conda create --name tigre_env -c anaconda -c ccpi -c conda-forge  python tigre simpleitk ipykernel opencv astropy tomopy nibabel scikit-image scikit-learn scipy tqdm scikit-learn-intelex jupyter ipywidgets
conda activate tigre_env

conda list --export > conda-package-list.txt
conda create -n tigre_env --file conda-package-list.txt
'''

import json
import math
import multiprocessing
import os
import sys
from __future__ import division


import cv2
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import tomopy
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans
from PIL import Image
from scipy import interpolate
from scipy.ndimage import median_filter
from scipy.signal import medfilt2d
from skimage.registration import phase_cross_correlation
from tqdm import trange, tqdm
from typing import List
import SimpleITK as sitk


import tigre
import tigre.algorithms as algs
from tigre.utilities.geometry import Geometry

import shared_functions as s

kernel = Gaussian2DKernel(x_stddev=2)

drive = 'f:\\'
base_folder = os.path.join(drive, 'jasper', 'data', '2022090_Alignment_oldplate_2')
base_json_file = os.path.join(base_folder, 'scan_settings.json')
results_folder = os.path.join(base_folder, 'results')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)


# Make a list of globals for the reconstruction setting, and log them in a json file
gReconParams = dict()

gReconParams['pixels'] = 512  # (pixels)
gReconParams['pixel_pitch'] = 0.055  # (mm)
gReconParams['fill_gap'] = True
gReconParams['median_filter'] = False
gReconParams['bad_pixel_correction'] = True
gReconParams['recon_voxels'] = (
    gReconParams['pixels'], gReconParams['pixels'], gReconParams['pixels'])  # number of voxels (vx)
 
''' TODO These should really be read from the JSON file! '''
gReconParams['distance_source_detector'] = 188.347  # 9+9+30+100+30+9+1.347 (mm)
gReconParams['z_stage_distance_mm'] = s.get_sample_z_from_first_scan_json(base_json_file) # Varies between 0 and 100 mm
gReconParams['distance_object_detector'] = 30 + 3 + \
    gReconParams['z_stage_distance_mm'] + 9+1.347  # (mm)
gReconParams['detector_rotation'] = (math.radians(-0.3), 0., 0.)  # (mm)

assert gReconParams['z_stage_distance_mm'] < 100 and gReconParams['z_stage_distance_mm'] >= 0


DSD = gReconParams['distance_source_detector']
DSO = DSD - gReconParams['distance_object_detector']
# a = 2 * 512 * 0.055 / (((DSD-DSO) / DSO) + 1) #TODO have to explain the numbers
a = 512 * 0.055 / (((DSD-DSO) / DSO) + 1) #TODO have to explain the numbers ::: For 20220822_ffpe_whateverBreast
gReconParams['recon_size'] = (a, a, a)
# gReconParams['recon_size'] = (512*0.055,512*0.055,512*0.055)

print(gReconParams)

# 20220805_tumourWhateverBreast
# centre_of_rotation_offset_x_mm = -0.27  # s.find_optimal_offset(gReconParams, spectral_projs_th0[1, :, :, :], angles, detector_x_offsets, detector_y_offsets, stage_offset=0, search_range=25)
# centre_of_rotation_offset_y_mm = -0.24

# 20220822_ffpe_WhateverBreast
# centre_of_rotation_offset_x_mm = 0.16885  # s.find_optimal_offset(gReconParams, spectral_projs_th0[1, :, :, :], angles, detector_x_offsets, detector_y_offsets, stage_offset=0, search_range=25)
# centre_of_rotation_offset_y_mm = 0.0

# 20220727_HamnCheeseseries_M
centre_of_rotation_offset_x_mm = 0.43725  # s.find_optimal_offset(gReconParams, spectral_projs_th0[1, :, :, :], angles, detector_x_offsets, detector_y_offsets, stage_offset=0, search_range=25)
centre_of_rotation_offset_y_mm = 0

print(f'centre_of_rotation_offset_x_mm = {centre_of_rotation_offset_x_mm} (mm)')
print(f'centre_of_rotation_offset_y_mm = {centre_of_rotation_offset_y_mm} (mm)')

{'pixels': 512, 'pixel_pitch': 0.055, 'fill_gap': True, 'median_filter': False, 'bad_pixel_correction': True, 'recon_voxels': (512, 512, 512), 'distance_source_detector': 188.347, 'z_stage_distance_mm': 0.0, 'distance_object_detector': 43.347, 'detector_rotation': (-0.005235987755982988, 0.0, 0.0), 'recon_size': (21.679134788448977, 21.679134788448977, 21.679134788448977)}
centre_of_rotation_offset_x_mm = 0.43725 (mm)
centre_of_rotation_offset_y_mm = 0 (mm)


In [2]:
spectral_projs_th0, spectral_open_th0, spectral_projs_th1, spectral_open_th1, th0_list, th1_list, exp_time, angles, z_offset, detector_x_offsets, detector_y_offsets, th0_dacs_list, th1_dacs_list = \
    s.load_or_generate_data_arrays(base_json_file, base_folder, results_folder, gReconParams)

Making new numpy files, should take ~4.5 minutes. At least one file was missing :( 


100%|██████████| 1/1 [00:50<00:00, 50.22s/it]


In [ ]:
print(spectral_projs_th0.shape)

In [3]:
open_mean_th0 = np.mean(spectral_open_th0, axis=1)
open_mean_th1 = np.mean(spectral_open_th1, axis=1)
print(open_mean_th0.shape)
ofc = -np.log(spectral_projs_th0[0,:,:,:]/open_mean_th0[0,:,:])
# ofc_bpc = s.save_and_or_load_npy_files(
#             results_folder, f'th{0}_bpc.npy', lambda: s.generate_bad_pixel_corrected_array(ofc, gReconParams))
stage_offset =  0.0
centre_of_rotation_offset_x_mm = 0.264#s.find_optimal_offset(gReconParams, ofc, angles, detector_x_offsets, detector_y_offsets, stage_offset=stage_offset, search_range=20)
print(centre_of_rotation_offset_x_mm)
gReconParams['detector_rotation'] = (math.radians(0.3), 0., 0.)  # (mm)
img_th0, geo = s.recon_scan(gReconParams, ofc, angles, detector_x_offsets,
                         detector_y_offsets, centre_of_rotation_offset_x_mm, centre_of_rotation_offset_y_mm, True)
    
ni_img = s.make_Nifti1Image(img_th0, geo.dVoxel)
s.save_array(results_folder, 'Recon_th0_.nii', ni_img)

(1, 512, 512)
0.264
dsd:  188.347 dso:  145.0
Saving Nifti array file: f:\jasper\data\2022090_Alignment_oldplate_2\results\Recon_th0_.nii
